# IMPORTS

In [160]:
import pandas as pd
import numpy as np
import os
from decimal import Decimal
import pickle
from pathlib import Path
from datetime import timedelta
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

# SETUP

In [161]:
dir_tree_util_path = os.path.join("utils", "dir_tree.py")
exec(open(dir_tree_util_path).read())

# INPUTS

In [70]:
# Paths
path_input = PROJECT_DIRS["DADOS_VEQ_ANTT_DIR"]
path_output = PROJECT_DIRS["DADOS_TRAFEGO_DIR"]
path_output_anual = PROJECT_DIRS["DADOS_ANUAL_DIR"]
path_output_trimestral = PROJECT_DIRS["DADOS_TRIMESTRAL_DIR"]
path_output_mensal = PROJECT_DIRS["DADOS_MENSAL_DIR"]
periodo = list(range(2010, 2025))

# Criando um dataframe consolidado com os dados de tráfego

In [71]:
# arquivos dos dados
arquivos = os.listdir(path_input)

In [72]:
df_trafego = pd.DataFrame()
for ano in periodo:
    arquivo = [a for a in arquivos if a[:-4].endswith(str(ano))][0]
    df_ano = pd.read_csv(os.path.join(path_input, arquivo), sep=';', encoding='cp1252', low_memory=False)
    df_trafego = pd.concat([df_trafego,df_ano])

# EDA Inicial

In [73]:
df_trafego.head()

,concessionaria,mes_ano,sentido,praca,categoria,tipo_de_veiculo,volume_total,multiplicador_de_tarifa,volume_veiculo_equivalente,tipo_de_cobranca
0,RODOVIA DO AÇO,01-01-2010,Decrescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,44146,1,44146,NaN
1,RODOVIA DO AÇO,01-01-2010,Crescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,35771,1,35771,NaN
2,RODOVIA DO AÇO,01-02-2010,Decrescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,33455,1,33455,NaN
3,RODOVIA DO AÇO,01-02-2010,Crescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,27109,1,27109,NaN
4,RODOVIA DO AÇO,01-03-2010,Decrescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,31677,1,31677,NaN


In [74]:
# mes_ano na verdade é uma data
df_trafego['mes_ano'].sample(9, random_state=3)

1057786    25/01/2024
2065656    17/03/2024
1831664    17/04/2024
34616      01/06/2021
242257     11/03/2024
764703     15/04/2024
13788      01/11/2019
86407      01/12/2021
41039      30/01/2024
Name: mes_ano, dtype: object

In [75]:
df_trafego.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000353 entries, 0 to 2329761
Data columns (total 10 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   concessionaria              object
 1   mes_ano                     object
 2   sentido                     object
 3   praca                       object
 4   categoria                   object
 5   tipo_de_veiculo             object
 6   volume_total                object
 7   multiplicador_de_tarifa     object
 8   volume_veiculo_equivalente  object
 9   tipo_de_cobranca            object
dtypes: object(10)
memory usage: 251.8+ MB


In [76]:
for col in df_trafego:
    print(f"{col}: {df_trafego[col].isna().sum()}")

concessionaria: 0
mes_ano: 0
sentido: 0
praca: 0
categoria: 0
tipo_de_veiculo: 0
volume_total: 344
multiplicador_de_tarifa: 0
volume_veiculo_equivalente: 0
tipo_de_cobranca: 74780


In [77]:
df_trafego[df_trafego['volume_total'].isna()].sample(9)

,concessionaria,mes_ano,sentido,praca,categoria,tipo_de_veiculo,volume_total,multiplicador_de_tarifa,volume_veiculo_equivalente,tipo_de_cobranca
130918,ECOVIAS DO CERRADO,01/11/2023,Decrescente,P4 - SANTA VITÓRIA,Veículo Comercial Acima 10 eixos,Comercial,NaN,"10,00","0,00",N/I
122983,ECOPONTE,01/05/2023,Decrescente,"Praça 01 BR-101/RJ km 322,20",Veículo Comercial Acima 10 eixos,Comercial,NaN,"10,00","0,00",N/I
122955,ECOPONTE,01/03/2023,Decrescente,"Praça 01 BR-101/RJ km 322,20",Veículo Comercial Acima 10 eixos,Comercial,NaN,"10,00","0,00",N/I
134585,RIOSP,01/02/2023,Crescente,"Praça 08 BR-116/SP km 205,00",Categoria 4,Comercial,NaN,"3,00","0,00",N/I
134593,RIOSP,01/02/2023,Crescente,"Praça 08 BR-116/SP km 205,00",Categoria 9,Moto,NaN,"0,50","0,00",N/I
131165,ECOVIAS DO CERRADO,01/12/2023,Crescente,"P7 - Jataí BR 364/GO km 156,550",Veículo Comercial Acima 10 eixos,Comercial,NaN,"10,00","0,00",N/I
126172,ECOSUL,01/12/2023,Decrescente,"Praça 04 BR-392/RS km 52,30",Categoria 9,Moto,NaN,"0,50","0,00",N/I
126072,ECOSUL,01/11/2023,Decrescente,"Praça 04 BR-392/RS km 52,30",Categoria 9,Moto,NaN,"0,50","0,00",N/I
125182,ECOSUL,01/02/2023,Crescente,"Praça 05 BR-392/RS km 111,47",Categoria 9,Moto,NaN,"0,50","0,00",N/I


In [78]:
df_trafego[df_trafego['volume_total'].isna()]['volume_veiculo_equivalente'].unique()

array(['0,00'], dtype=object)

In [79]:
df_trafego['mes_ano'].str[:2].unique()

array(['01', '14', '28', '31', '30', '02', '03', '04', '05', '06', '07',
       '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27', '29'], dtype=object)

In [80]:
df_trafego['mes_ano'].str[3:5].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12'], dtype=object)

In [81]:
df_trafego['tipo_de_cobranca'].unique()

array([nan, 'N/I', 'Manual', 'Automática', 'Mista'], dtype=object)

In [82]:
df_trafego['volume_total'].isna().any()

np.True_

In [83]:
df_trafego['volume_total'].isna().sum()

np.int64(344)

In [84]:
df_trafego['multiplicador_de_tarifa'].unique()

array(['1', '2', '1,5', '3', '4', '5', '6', '0,5', '7', '8', '9', '10',
       1.0, 2.0, 1.5, 3.0, 4.0, 5.0, 6.0, 0.5, 7.0, 8.0, 9.0, 10.0,
       '2,00', '3,00', '4,00', '5,00', '6,00', '0,50', '1,00', '1,50',
       '7,00', '8,00', '9,00', '10,00', '0,00', '11,00', '13,00', '15,00',
       '12,00', '14,00', '18,00', '20,00', '16,00', '17,00', '19,00'],
      dtype=object)

# Tratamento dos dados

In [85]:
# Colunas de ano, mes e dia
df_trafego['year'] = df_trafego['mes_ano'].str[-4:].astype('int') 
df_trafego['month'] = df_trafego['mes_ano'].str[3:5].astype('int')
df_trafego['day'] = df_trafego['mes_ano'].str[:2].astype('int') 

In [86]:
# Recriando a coluna de data, agora como datetime ao invés de string. Tem que ser em ingles pq a função do pandas exige
# OBS: essa coluna está cagada, as vezes temos a data completa, as vezes apenas um registro consolidando o mês!
df_trafego['data'] = pd.to_datetime(df_trafego[['year', 'month', 'day']]).dt.date
# criando uma coluna de mes_ano
df_trafego['mes_ano'] = pd.to_datetime(df_trafego[['year', 'month', 'day']]).dt.to_period('M')

In [87]:
# OBS: para algumas concessionarias ao menos, os dados a nível de data não são confiáveis

df_trafego.query("concessionaria == 'CRO' and mes_ano == @pd.to_datetime('2015-09').to_period('M')")['data'].unique()


# Os dados com início com 2010-01-01 estão ok.

array([datetime.date(2015, 9, 1)], dtype=object)

In [88]:
# preenchendo os NaNs, qdo aplicável:
df_trafego['volume_total'] = df_trafego['volume_total'].fillna(0) #o volume equivalente é zero sempre que volume_total é zero

In [89]:
# Convertendo os tipos de dados para numéricos:
df_trafego['volume_total'] = df_trafego['volume_total'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
df_trafego['volume_total'] = df_trafego['volume_total'].apply(lambda x: x.split('.')[0] if type(x) == str else x)
df_trafego['volume_total'] = df_trafego['volume_total'].astype('int')

df_trafego['volume_veiculo_equivalente'] = df_trafego['volume_veiculo_equivalente'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
df_trafego['volume_veiculo_equivalente'] = df_trafego['volume_veiculo_equivalente'].astype('float') # existe multiplicador de tarifa fracionário

In [90]:
# convertendo categoria para string (do contrário não salva para parquet):
df_trafego['categoria'] = df_trafego['categoria'].astype('string')

In [91]:
# convertendo o multiplicador de tarifa de string para decimal (p/manter a precisao):
df_trafego['multiplicador_de_tarifa'] = df_trafego['multiplicador_de_tarifa'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
df_trafego['multiplicador_de_tarifa'] = df_trafego['multiplicador_de_tarifa'].apply(Decimal)

In [92]:
# O tipo de tráfego ora está em maisúcula, ora em minúscula.
# Colocando tudo p/maiúscula
df_trafego['tipo_de_veiculo'] = df_trafego['tipo_de_veiculo'].str.upper()

In [93]:
# algumas concessionárias estão ora em maiúsculas ora em minúsculas
# colocando tudo para maiúscula
df_trafego['concessionaria'] = df_trafego['concessionaria'].str.upper()

In [94]:
df_trafego.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000353 entries, 0 to 2329761
Data columns (total 14 columns):
 #   Column                      Dtype    
---  ------                      -----    
 0   concessionaria              object   
 1   mes_ano                     period[M]
 2   sentido                     object   
 3   praca                       object   
 4   categoria                   string   
 5   tipo_de_veiculo             object   
 6   volume_total                int64    
 7   multiplicador_de_tarifa     object   
 8   volume_veiculo_equivalente  float64  
 9   tipo_de_cobranca            object   
 10  year                        int64    
 11  month                       int64    
 12  day                         int64    
 13  data                        object   
dtypes: float64(1), int64(4), object(7), period[M](1), string(1)
memory usage: 343.4+ MB


In [95]:
df_trafego['year'].unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022, 2023, 2024])

In [96]:
df_trafego.query("year == 2024")['month'].unique()

array([1, 2, 3, 4, 5, 6])

## Criando coluna desambiguando categorias comercial x passeio

In [97]:
df_trafego['tipo_de_veiculo'].unique()

array(['PASSEIO', 'COMERCIAL', 'MOTO', 'VEÍCULO PEQUENO'], dtype=object)

In [98]:
df_trafego["TIPO_TRAFEGO"] = df_trafego['tipo_de_veiculo'].apply(lambda x: x if x == "COMERCIAL" else "PASSEIO")

In [99]:
df_trafego["TIPO_TRAFEGO"].unique()

array(['PASSEIO', 'COMERCIAL'], dtype=object)

## Criando colunas com trafégo em Veqs para comercial e passeio

In [100]:
df_trafego = df_trafego.rename(columns={'volume_veiculo_equivalente':'VEQS_TOTAL'})
df_trafego['VEQS_COMERCIAL'] = np.where(
    df_trafego['TIPO_TRAFEGO'] == 'COMERCIAL',
    df_trafego['VEQS_TOTAL'],
    0
)
df_trafego['VEQS_PASSEIO'] = np.where(
    df_trafego['TIPO_TRAFEGO'] == 'PASSEIO',
    df_trafego['VEQS_TOTAL'],
    0
)

# Criando dataframe com o período inicial e final dos dados por concessionária

In [101]:
df_periodos = df_trafego.groupby('concessionaria').agg(
    data_inicial=('data', 'min'),
    data_final=('data', 'max')
).reset_index()
df_periodos['data_inicial'] = pd.to_datetime(df_periodos['data_inicial']).dt.date
df_periodos['data_final'] = pd.to_datetime(df_periodos['data_final']).dt.date

In [102]:
df_periodos.index = df_periodos['concessionaria']
df_periodos = df_periodos.drop(columns=['concessionaria'])

In [103]:
df_periodos = df_periodos.sort_values(by='data_inicial', ascending=False)

# Criando dataframes agrupando VEQs por ANO/concessionaria/tipo de trafego

In [104]:
cols_veqs = ['VEQS_COMERCIAL','VEQS_PASSEIO']

In [105]:
dict_veqs_anual = {}
for col in cols_veqs:
    dict_veqs_anual[col] = df_trafego.pivot_table(
        index='concessionaria',
        columns='year',
        values=col,
        aggfunc='sum'
        ).T

In [106]:
def filter_incomplete_years(df_trafego, df_periodos):
    df = df_trafego.copy()
    for conc in df:
        data_inicial = df_periodos.loc[conc,'data_inicial']
        data_final = df_periodos.loc[conc,'data_final']
        # print(data_inicial, data_final)
        years_drop = []
        if (data_inicial.day != 1) or (data_inicial.month != 1):
            years_drop.append(data_inicial.year)
        if (data_final.day != 31) or (data_final.month != 12):
            years_drop.append(data_final.year)
        # print(list(set(years_drop)))
        years_drop = list(set(years_drop))
        for year in df.index:
            if year in years_drop:
                df.loc[year, conc] = None
    return df    

In [107]:
# retirando os anos incompletos
def filter_incomplete_years(df_trafego, df_periodos):
    df = df_trafego.copy()
    for conc in df:
        data_inicial = df_periodos.loc[conc,'data_inicial']
        data_final = df_periodos.loc[conc,'data_final']
        # print(data_inicial, data_final)
        years_drop = []
        if (data_inicial.day != 1) or (data_inicial.month != 1):
            years_drop.append(data_inicial.year)
        if (data_final.day != 31) or (data_final.month != 12):
            years_drop.append(data_final.year)
        # print(list(set(years_drop)))
        years_drop = list(set(years_drop))
        for year in df.index:
            if year in years_drop:
                df.loc[year, conc] = None
    return df    

In [108]:
for veq, df in dict_veqs_anual.items():
    dict_veqs_anual[veq] = filter_incomplete_years(df, df_periodos)

In [109]:
dict_veqs_anual['VEQS_COMERCIAL'].head()

concessionaria,AUTOPISTA FERNÃO DIAS,AUTOPISTA FLUMINENSE,AUTOPISTA LITORAL SUL,AUTOPISTA PLANALTO SUL,AUTOPISTA REGIS BITTENCOURT,CONCEBRA,CONCEPA,CONCER,CRO,CRT,ECO050,ECO101 CONCESSIONARIA DE RODOVIAS S/A,ECOPONTE,ECORIOMINAS,ECOSUL,ECOVIAS DO ARAGUAIA,ECOVIAS DO CERRADO,HOLDING DO SISTEMA RODOVIARIO RIO - SAO PAULO S.A.,MSVIA,NOVADUTRA,RIOSP,RODOVIA DO AÇO,TRANSBRASILIANA,VIA 040,VIA BAHIA,VIA BRASIL,VIA COSTEIRA,VIA SUL
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010,92215435.0,23879336.0,69931704.0,19554781.0,114884661.0,NaN,14096537.0,13109979.0,NaN,8563340.0,NaN,NaN,NaN,NaN,14426940.0,NaN,NaN,NaN,NaN,116380982.0,NaN,12887198.0,15514995.0,NaN,NaN,NaN,NaN,NaN
2011,111335578.0,26387101.0,74189265.0,20394485.0,122893798.0,NaN,14804840.0,14804816.0,NaN,9264103.0,NaN,NaN,NaN,NaN,16168263.0,NaN,NaN,NaN,NaN,125958069.0,NaN,12595032.0,16821634.0,NaN,NaN,NaN,NaN,NaN
2012,115803681.0,27188502.0,77030428.0,19941264.0,123630694.0,NaN,15742320.0,15723249.0,NaN,9892007.0,NaN,NaN,NaN,NaN,15672205.0,NaN,NaN,NaN,NaN,122928827.0,NaN,11550846.0,18435974.0,NaN,NaN,NaN,NaN,NaN
2013,117936734.0,26658829.0,78950113.0,20865630.0,126658897.0,NaN,17190745.0,16066640.0,NaN,9813665.0,NaN,NaN,NaN,NaN,19186163.0,NaN,NaN,NaN,NaN,123410424.0,NaN,12922325.0,18752871.0,NaN,67484476.0,NaN,NaN,NaN
2014,114170690.0,25867004.0,98370521.0,21843099.0,123633793.0,NaN,17876501.0,15291939.0,NaN,10186433.0,NaN,NaN,NaN,NaN,19934187.0,NaN,NaN,NaN,NaN,119182763.0,NaN,12850352.0,18614344.0,NaN,69684068.0,NaN,NaN,NaN


# Criando dataframes agrupando VEQs por MÊS/concessionaria/tipo de trafego

In [110]:
dict_veqs_mensal = {}
for col in cols_veqs:
    dict_veqs_mensal[col] = df_trafego.pivot_table(
        index='concessionaria',
        columns='mes_ano',
        values=col,
        aggfunc='sum'
        ).T
    dict_veqs_mensal[col].index = dict_veqs_mensal[col].index.to_timestamp().to_period('M')

In [111]:
def filter_incomplete_months(df_trafego, df_periodos):
    df = df_trafego.copy()

    for conc in df:
        data_inicial = df_periodos.loc[conc, 'data_inicial']
        data_final = df_periodos.loc[conc, 'data_final']
        
        # List to keep track of months to drop
        months_drop = []

        # Check if the starting month is incomplete
        if data_inicial.day != 1:
            # Drop the starting month if not starting from the 1st day
            incomplete_start = pd.to_datetime(data_inicial).to_period('M')
            months_drop.append(incomplete_start)

        # Check if the ending month is incomplete
        if data_final != (data_final + pd.offsets.MonthEnd(0)):
            # Drop the ending month if not ending on the last day
            incomplete_end = pd.to_datetime(data_final).to_period('M')
            months_drop.append(incomplete_end)

        # Remove duplicate dates (if any)
        months_drop = list(set(months_drop))

    # Removendo os meses
        for month in df.index:
            if month in months_drop:
                df.loc[month, conc] = None         
    return df  

In [112]:
for veq, df in dict_veqs_mensal.items():
    dict_veqs_mensal[veq] = filter_incomplete_months(df, df_periodos)

In [113]:
# removendo o primeiro mês das que não iniciam em jan/10 (por algum erro dos dados, está incompleto)
def remove_first_month_post_2010(df_mensal, mes_nao_remover):
    for col in df_mensal:
        first_non_nan = df_mensal[col].first_valid_index()
        # print(col, first_non_nan)
        # print(col, df_mensal.loc[first_non_nan, col])
        if first_non_nan != pd.to_datetime(mes_nao_remover).to_period('M'):
            df_mensal.loc[first_non_nan, col] = None
        # print(col, df_mensal.loc[first_non_nan, col])
    return df_mensal

In [114]:
for veq, df in dict_veqs_mensal.items():
    dict_veqs_mensal[veq] = remove_first_month_post_2010(df, '2010-01')

In [115]:
dict_veqs_mensal['VEQS_COMERCIAL'].head()

concessionaria,AUTOPISTA FERNÃO DIAS,AUTOPISTA FLUMINENSE,AUTOPISTA LITORAL SUL,AUTOPISTA PLANALTO SUL,AUTOPISTA REGIS BITTENCOURT,CONCEBRA,CONCEPA,CONCER,CRO,CRT,ECO050,ECO101 CONCESSIONARIA DE RODOVIAS S/A,ECOPONTE,ECORIOMINAS,ECOSUL,ECOVIAS DO ARAGUAIA,ECOVIAS DO CERRADO,HOLDING DO SISTEMA RODOVIARIO RIO - SAO PAULO S.A.,MSVIA,NOVADUTRA,RIOSP,RODOVIA DO AÇO,TRANSBRASILIANA,VIA 040,VIA BAHIA,VIA BRASIL,VIA COSTEIRA,VIA SUL
mes_ano,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01,6190339.0,1815015.0,5183361.0,1464018.0,8261978.0,NaN,1156902.0,990659.0,NaN,653385.0,NaN,NaN,NaN,NaN,902674.0,NaN,NaN,NaN,NaN,7602748.0,NaN,986812.0,1088796.0,NaN,NaN,NaN,NaN,NaN
2010-02,5924839.0,1708351.0,5020757.0,1481465.0,7872436.0,NaN,1063578.0,905443.0,NaN,612653.0,NaN,NaN,NaN,NaN,850322.0,NaN,NaN,NaN,NaN,7065494.0,NaN,929227.0,1060479.0,NaN,NaN,NaN,NaN,NaN
2010-03,7003056.0,2018155.0,5864833.0,1701691.0,9965511.0,NaN,1221469.0,1114422.0,NaN,726098.0,NaN,NaN,NaN,NaN,1239520.0,NaN,NaN,NaN,NaN,9025403.0,NaN,1165079.0,1274481.0,NaN,NaN,NaN,NaN,NaN
2010-04,6875269.0,1752751.0,5433517.0,1627705.0,9190180.0,NaN,1115475.0,1007970.0,NaN,625073.0,NaN,NaN,NaN,NaN,1676946.0,NaN,NaN,NaN,NaN,9694048.0,NaN,1100444.0,1198545.0,NaN,NaN,NaN,NaN,NaN
2010-05,7518015.0,2005884.0,5796304.0,1746326.0,9827356.0,NaN,1155528.0,1084941.0,NaN,705851.0,NaN,NaN,NaN,NaN,1416282.0,NaN,NaN,NaN,NaN,10581017.0,NaN,1176777.0,1334414.0,NaN,NaN,NaN,NaN,NaN


# Criando dataframes agrupando VEQs por TRIMESTRE/concessionaria/tipo de trafego

In [146]:
def group_by_tri(df_monthly):
    df_tri = df_monthly.copy()
    
    # # Ensure the index is a datetime object
    df_tri.index = df_tri.index.to_timestamp()

    # Ensure the index is a datetime object
    # df_tri.index = pd.to_datetime(df_tri.index)  # Convert index to datetime if not already

    # Create a column for the quarter-year format
    df_tri['trimestre'] = df_tri.index.to_period('Q')

    # Identify quarters with any NaN values
    cols = [col for col in df_tri.columns if col != 'trimestre']
    for col in cols:
        # Find quarters where there is at least one NaN value in the column
        quarters_with_nan = df_tri.groupby('trimestre')[col].apply(lambda x: x.isna().any())
        
        # Filter only quarters that have NaN
        quarters_with_nan = quarters_with_nan[quarters_with_nan].index

        # Set all values in those quarters to NaN for the column
        df_tri.loc[df_tri['trimestre'].isin(quarters_with_nan), col] = None

    # Identify incomplete quarters in the index itself
    incomplete_quarters = df_tri.groupby('trimestre').size()
    incomplete_quarters = incomplete_quarters[incomplete_quarters < 3].index

    # Remove incomplete quarters
    df_tri = df_tri[~df_tri['trimestre'].isin(incomplete_quarters)]

    # Drop the helper column
    df_tri = df_tri.drop(columns=['trimestre'])

    # Group by trimester (quarter) and sum
    df_tri = df_tri.groupby(df_tri.index.to_period('Q')).sum()

    df_tri.index.name = 'trimestre'
    
    df_tri = df_tri.replace(0, np.nan) # colocando zero como NaN

    return df_tri

In [147]:
# def group_by_tri(df_monthly):
#     df_tri = df_monthly.copy()
   
#     # Ensure the index is a datetime object
#     df_tri.index = df_tri.index.to_timestamp()
    
#     # Create a column for the quarter-year format
#     df_tri['trimestre'] = df_tri.index.to_period('Q')
    
#     # Identify quarters with any NaN values
#     cols = [col for col in df_tri.columns if col != 'trimestre']
#     for col in cols:
#         # Find quarters where there is at least one NaN value in the column
#         quarters_with_nan = df_tri.groupby('trimestre')[col].apply(lambda x: x.isna().any())
        
#         # Filter only quarters that have NaN
#         quarters_with_nan = quarters_with_nan[quarters_with_nan].index

#         # Set all values in those quarters to zero for the column
#         df_tri.loc[df_tri['trimestre'].isin(quarters_with_nan), col] = None

#     # Drop the helper column
#     df_tri = df_tri.drop(columns=['trimestre'])

#     # Group by trimester (quarter) and sum
#     df_tri = df_tri.groupby(df_tri.index.to_period('Q')).sum()
    
#     df_tri.index.name = 'trimestre'
#     # df_tri.index = df_tri.index.to_period('Q')

#     return df_tri


In [148]:
dict_veqs_tri = {}
for veq, df in dict_veqs_mensal.items():
    dict_veqs_tri[veq] = group_by_tri(df)

In [150]:
dict_veqs_tri['VEQS_COMERCIAL'].head()

concessionaria,AUTOPISTA FERNÃO DIAS,AUTOPISTA FLUMINENSE,AUTOPISTA LITORAL SUL,AUTOPISTA PLANALTO SUL,AUTOPISTA REGIS BITTENCOURT,CONCEBRA,CONCEPA,CONCER,CRO,CRT,ECO050,ECO101 CONCESSIONARIA DE RODOVIAS S/A,ECOPONTE,ECORIOMINAS,ECOSUL,ECOVIAS DO ARAGUAIA,ECOVIAS DO CERRADO,HOLDING DO SISTEMA RODOVIARIO RIO - SAO PAULO S.A.,MSVIA,NOVADUTRA,RIOSP,RODOVIA DO AÇO,TRANSBRASILIANA,VIA 040,VIA BAHIA,VIA BRASIL,VIA COSTEIRA,VIA SUL
trimestre,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010Q1,19118234.0,5541521.0,16068951.0,4647174.0,26099925.0,NaN,3441949.0,3010524.0,NaN,1992136.0,NaN,NaN,NaN,NaN,2992516.0,NaN,NaN,NaN,NaN,23693645.0,NaN,3081118.0,3423756.0,NaN,NaN,NaN,NaN,NaN
2010Q2,21643553.0,5701041.0,16910430.0,4987917.0,28377179.0,NaN,3380149.0,3157629.0,NaN,2015505.0,NaN,NaN,NaN,NaN,4325632.0,NaN,NaN,NaN,NaN,30087408.0,NaN,3342063.0,3786746.0,NaN,NaN,NaN,NaN,NaN
2010Q3,24157523.0,6183923.0,18332337.0,4948655.0,29787088.0,NaN,3510893.0,3418319.0,NaN,2221628.0,NaN,NaN,NaN,NaN,3666469.0,NaN,NaN,NaN,NaN,31267360.0,NaN,3246286.0,4151907.0,NaN,NaN,NaN,NaN,NaN
2010Q4,27296125.0,6452851.0,18619986.0,4971035.0,30620469.0,NaN,3763546.0,3523507.0,NaN,2334071.0,NaN,NaN,NaN,NaN,3442323.0,NaN,NaN,NaN,NaN,31332569.0,NaN,3217731.0,4152586.0,NaN,NaN,NaN,NaN,NaN
2011Q1,26160702.0,6215518.0,17250630.0,5212567.0,29285468.0,NaN,3540257.0,3533875.0,NaN,2188312.0,NaN,NaN,NaN,NaN,3626090.0,NaN,NaN,NaN,NaN,29355052.0,NaN,2984654.0,3786002.0,NaN,NaN,NaN,NaN,NaN


# Salvando p/parquet

In [151]:
# dataframe total
# df_trafego.to_parquet(os.path.join(path_dados_derivados,'df_trafego.parquet'))
df_trafego.to_parquet(path_output /'df_trafego.parquet')

In [152]:
# data inicial e final
# OBS: alguns períodos finais bugados
df_periodos.to_parquet(path_output /'df_periodos.parquet')

In [153]:
# dicionário com os dados de tráfego anuais:
for veq in dict_veqs_anual:
    dict_veqs_anual[veq].to_parquet(path_output_anual / f'df_{veq}_anual.parquet')

In [154]:
# dicionário com os dados de tráfego mensais:
for veq in dict_veqs_mensal:
    dict_veqs_mensal[veq].to_parquet(path_output_mensal / f'df_{veq}_mensal.parquet')

In [155]:
# dicionário com os dados de tráfego trimestrais:
for veq in dict_veqs_tri:
    dict_veqs_tri[veq].to_parquet(path_output_trimestral / f'df_{veq}_trimestral.parquet')

# Salvando p/Excel

In [156]:
# Data inicial e final de tráfego por concessionária:
df_periodos.to_excel(path_output / 'periodo_dados_trafego.xlsx')

In [157]:
# dados de tráfego anuais
with pd.ExcelWriter(path_output_anual / 'dados_trafego_ANTT_anual.xlsx') as writer:
    # df_periodos.to_excel(writer, sheet_name="periodo_concessionarias")
    for veq in dict_veqs_anual:
        dict_veqs_anual[veq].to_excel(writer, sheet_name=f"{veq}")

In [158]:
# dados de tráfego mensais
with pd.ExcelWriter(path_output_mensal / 'dados_trafego_ANTT_mensal.xlsx') as writer:
    # df_periodos.to_excel(writer, sheet_name="periodo_concessionarias")
    for veq,df in dict_veqs_mensal.items():
        df_export = df.copy()
        df_export = df_export.T
        new_index = [col.strftime('%Y-%m-%d') for col in df_export.columns]
        df_export.columns = new_index
        df_export = df_export.T
        df_export.to_excel(writer, sheet_name=f"{veq}")

In [159]:
# dados de tráfego trimestrais
with pd.ExcelWriter(path_output_trimestral / 'dados_trafego_ANTT_trimestral.xlsx') as writer:
    # df_periodos.to_excel(writer, sheet_name="periodo_concessionarias")
    for veq,df in dict_veqs_tri.items():
        df_export = df.copy()
        df_export = df_export.T
        new_index = [col.strftime('%Y-%m-%d') for col in df_export.columns]
        df_export.columns = new_index
        df_export = df_export.T
        df_export.to_excel(writer, sheet_name=f"{veq}")